In [1]:
from dotenv import load_dotenv
import os
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel, set_tracing_disabled


load_dotenv()
QWEN_API_KEY = os.getenv("QWEN_API_KEY")
QWEN_BASE_URL = os.getenv("QWEN_BASE_URL")
QWEN_MODEL_NAME = os.getenv("QWEN_MODEL_NAME")
client = AsyncOpenAI(base_url=QWEN_BASE_URL, api_key=QWEN_API_KEY)
set_tracing_disabled(disabled=True)
qwen_model = OpenAIChatCompletionsModel(model=QWEN_MODEL_NAME, openai_client=client)

In [2]:
import json
from agents import Agent, Runner, function_tool

MEMORY_FILE = "memory.json"

def _load_memory():
    """Loads the memory from the JSON file."""
    try:
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

def _save_memory(data):
    """Saves the memory to the JSON file."""
    with open(MEMORY_FILE, "w") as f:
        json.dump(data, f, indent=2)


In [3]:
@function_tool
def save_memory(key: str, value: str) -> str:
    """
    Saves a value to the agent's persistent memory.
    Use this to remember specific facts, user preferences, or past interactions.
    For example: save_memory(key="user_name", value="Alex")
    """
    memory = _load_memory()
    memory[key] = value
    _save_memory(memory)
    return f"Successfully saved value for key: {key}"

@function_tool
def recall_memory(key: str) -> str:
    """
    Recalls a value from the agent's persistent memory using a key.
    Use this to retrieve information you have previously saved.
    For example: recall_memory(key="user_name")
    """
    memory = _load_memory()
    value = memory.get(key)
    if value:
        return f"Recalled value for key '{key}': {value}"
    return f"No value found for key: {key}"


In [4]:
memory_agent = Agent(
    name="Memory Agent",
    instructions=(
        "You are a helpful assistant with a long-term memory. "
        "When a user tells you something important to remember, use the save_memory tool. "
        "When you need to access past information, use the recall_memory tool. "
        "For example, if a user tells you their name, save it. If they ask you what their name is, recall it."
    ),
    tools=[save_memory, recall_memory],
    model=OpenAIChatCompletionsModel(model=QWEN_MODEL_NAME, openai_client=client),
)


In [ ]:
# First interaction: Save a fact
print("User: Please remember my name is Bob.")
result1 =await Runner.run(memory_agent,"Please remember my name is Bob.")

print(f"Agent's final output: {result1.final_output}")


User: Please remember my name is Bob.
Agent's final output: Hello, Bob! It's nice to meet you. How can I assist you today?


In [12]:
print(result1)

RunResult:
- Last agent: Agent(name="Memory Agent", ...)
- Final output (str):
    Hello, Bob! It's nice to meet you. How can I assist you today?
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [15]:
import pprint
pprint.pprint(result1.to_input_list())

[{'content': 'Please remember my name is Bob.', 'role': 'user'},
 {'arguments': '{"key": "user_name", "value": "Bob"}',
  'call_id': 'call_67f869a3b5994a908cec48',
  'id': '__fake_id__',
  'name': 'save_memory',
  'type': 'function_call'},
 {'call_id': 'call_67f869a3b5994a908cec48',
  'output': 'Successfully saved value for key: user_name',
  'type': 'function_call_output'},
 {'content': [{'annotations': [],
               'text': "Hello, Bob! It's nice to meet you. How can I assist "
                       'you today?',
               'type': 'output_text'}],
  'id': '__fake_id__',
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]


In [18]:
# Second interaction: Recall the fact
print("\nUser: What is my name?")
result2 =await Runner.run(memory_agent,"What is my name? Use the recall_memory tool to recall the user_name.")
# The agent will call recall_memory(key="user_name") and use the result
print(f"Agent's final output: {result2.final_output}")


User: What is my name?
Agent's final output: Your name is Bob. Let me know if there's anything else I can assist you with!


In [19]:
import pprint
pprint.pprint(result2.to_input_list())

[{'content': 'What is my name? Use the recall_memory tool to recall the '
             'user_name.',
  'role': 'user'},
 {'arguments': '{"key": "user_name"}',
  'call_id': 'call_a91be550fff448a0b8f272',
  'id': '__fake_id__',
  'name': 'recall_memory',
  'type': 'function_call'},
 {'call_id': 'call_a91be550fff448a0b8f272',
  'output': "Recalled value for key 'user_name': Bob",
  'type': 'function_call_output'},
 {'content': [{'annotations': [],
               'text': "Your name is Bob. Let me know if there's anything else "
                       'I can assist you with!',
               'type': 'output_text'}],
  'id': '__fake_id__',
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]
